## Chatper 4 분류

### 4. 랜덤 포레스트

#### 랜덤 포레스트의 개요 및 실습
랜덤 포레스트는 배깅(bagging)의 대표적인 알고리즘으로 속도도 빠르고 예측 성농도 좋다. 기반 알고리즘은 결정 트리로써 결정 트리의 쉽고 직관적인 장점을 그래도 가지고 있다.
랜덤 포레스트는 여러 개의 결정 트리 분류기가 전체 데이터에서 배깅 방식으로 각자의 데이터를 샘플링해 개별적으로 학습한 뒤 모든 분류기가 보팅을 통해 예측 결정을 하게 된다.
랜덤 포레스트는 개별적인 분류기의 기반 알고리즘은 결정 트리이지만, 개별 트리가 학습하는 데이터 세트는 전체 데이터 세트에서 일부가 중첩되게 샘플링된 데이터 세트이다. 이렇게 여러 데이터가 중첩되게 분리하는 것을 부트스트래핑(bootstrapping) 분할 방식이라고 한다. 랜덤 포레스트의 서브 세트 데이터의 수는 전체 데이터 세트의 수와 같지만 개별 데이터가 중첩되어 만들어진다.

In [1]:
# 랜덤 포레스트 실습을 위한 데이터셋 불러오기
import pandas as pd

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./data/human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./data/human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./data/human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./data/human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./data/human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 랜덤 포레스트 학습 및 별도의 테스트 세트로 예측 성능 평가
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('랜덤 포레스트 정확도 : {0:.4f}'.format(accuracy))

#### 랜덤 포레스트 하이퍼 파라미터 및 튜닝

일반적으로 트리 기반의 앙상블 알고리즘의 단점은 하이퍼 파라미터가 너무 많고, 그로 인해 튜닝을 위한 시간이 많이 소요된다는 점이다. 그럼에도 불구하고 예측 성능이 크게 좋아지는 경우는 많지 않다.
그나마 랜덤 포레스트는 적은 편에 속하며 결정 트리에서 사용하는 하이퍼 파라미터와 같은 파라미터가 대부분이다.

- n_estimators : 결정 트리의 개수를 지정, 디폴트는 10이며 많이 설정할수록 좋은 성능 기대 가능 vut 항상 향상은 아니며 학습 수행 시간도 늘어남
- max_features : 결정 트리의 max_features와 같다. 기본값은 'auto', 즉, 'sqrt'와 같다. 따라서 랜덤 포레스트의 트리를 분할하는 피처를 참조할 때 sqrt(전체 피처 개수)만큼만 참조
- max_depth나 min_smaples_leaf와 같은 결정 트리에서 과적합 개선을 위한 파라미터도 똑같이 사용 가능

In [ ]:
# GridSearchCV를 이용해 최적의 하이퍼 파라미터를 찾아보자.


from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}
# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=2, n_jobs=-1 )
grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
# n_estimators를 300으로 늘려 최적의 하이퍼 파라미터 조합과 RandomForestClassifier를 학습시킨 뒤
# 별도의 테스트 데이터 세트에서 예측 성능을 측정해보자.

rf_clf1 = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_leaf=8, \
                                 min_samples_split=8, random_state=0)
rf_clf1.fit(X_train , y_train)
pred = rf_clf1.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

In [ ]:
# RandomForestClassifier도 결정 트리와 같이 feature_importances_ 속성을 이용해 피처의 중요드를 알 수 있다.

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ftr_importances_values = rf_clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns  )
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)
plt.show()

### 5. GBM(Gradient Boosting Machine)

#### GBM의 개요 및 실습
부스팅 알고리즘은 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식이다.
대표적으로 AdaBoost와 그래디언트 부스트가 있다.
AdaBoost는 각 Step마다 잘못 예측한 데이터에 가중치를 부여하며 학습을 진행해 나가며 GBM과의 차이점은 GBM은 가중치 업데이트를 경사 하강버을 이용하는 것이 큰 차이이다. 오류 값은 실제 값 - 예측으로 계산한다.
GBM은 회귀도 가능하다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train , y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

일반적으로는 랜덤 포레스트보다 성능이 뛰어난 경우가 많다. 그러나 수행 시간이 오래 걸리며 하이퍼 파라미터 튜닝 노력도 필요하다.

#### GBM 하이퍼 파라미터 및 튜닝

n_estimators, max_depth, max_features와 같은 트리 기반 자체의 파라미터는 앞에서 나왔으니 생략

- loss : 경사 하강법에서 사용할 비용 함수 지정. 기본값은 'deviance'
- learning_rate : 학습률이며 0~1 값 지정 가능하다. 0.1이 기본값이다. 학습률의 특성 상 n_estimators와 상호 보완적으로 조합해 사용한다. (학습률 작게, n_estimators 크게 but 시간 오래걸림)
- n_estimators : weak learner의 개수이며 기본값은 100이다.
- subsample : weak learner가 사용하는 데이터의 샘플링 비율. 기본 값은 1이며 이는 전체 학습 데이터를 기반으로 학습한다는 의미이다. 과적합이 염려되면 1보다 작게 설정하자.

In [ ]:
# 하이퍼 파라미터 튜닝

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100, 500],
    'learning_rate' : [ 0.05, 0.1]
}
grid_cv = GridSearchCV(gb_clf , param_grid=params , cv=2 ,verbose=1)
grid_cv.fit(X_train , y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
# GridSearchCV를 이용하여 최적으로 학습된 estimator로 predict 수행. 
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))